In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
!pip install javalang


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
import javalang


Tokenizer

In [ ]:
li=list(javalang.tokenizer.tokenize(og))

In [ ]:
print(li[0])

Modifier "public" line 2, position 1


In [ ]:
#easy to fetch number of identifiers 

In [ ]:
for elem in li:
  if str(elem).split(" ")[0]=='Identifier':
    print(str(elem).split(" ")[1])

In [ ]:
for elem in li:
  print(str(elem).split(" ")[0],str(elem).split(" ")[1])

New Model

In [101]:
import pandas as pd
import re
import javalang
import numpy as np

In [103]:
class CodeAnalyzer:
    def __init__(self, code):
        self.code = code
        self.tree = javalang.parse.parse(self.code)
        self.method_name = []
        self.method_return_type = []
        self.loops = []
        self.controlStruct =[]
        self.nesting_depth = 0
        self.variable_name=[]
        self.variable_datatype=[]

    def analyze_method(self):
        for path,node in self.tree.filter(javalang.tree.MethodDeclaration):
          self.method_name.append(node.name)
          if(node.return_type):
            self.method_return_type.append(node.return_type.name)
          else:
            self.method_return_type.append(node.return_type)        

    def analyze_variables(self):
        for path,node in self.tree.filter(javalang.tree.VariableDeclaration):
          self.variable_datatype.append(node.type.name)
          for variable_declarator in node.declarators:
            self.variable_name.append(variable_declarator.name)

    def analyze_loops(self):
        for path,node in self.tree:
          if isinstance(node, javalang.tree.ForStatement):
                  self.loops.append('For')
          if isinstance(node, javalang.tree.DoStatement):
                  self.loops.append('Do')
          if isinstance(node, javalang.tree.WhileStatement):
                  self.loops.append('While')

    def analyze_control_structures(self):
        for path,node in self.tree:
          if isinstance(node, javalang.tree.IfStatement):
                  self.loops.append('If')
          if isinstance(node, javalang.tree.SwitchStatement):
                  self.loops.append('Switch')
            
    def analyze_nesting_depth(self):
      pass

    def find_loc(self):
      #no multiline comments
      clean_code=re.sub(r'/\*(.*?)\*/', '', self.code, flags=re.DOTALL)
      #no tabs
      clean_code=re.sub(r'\t',' ',clean_code)
      #no empty lines
      clean_code = re.sub(r'\n(\s*)\n', '\n', clean_code)
      clean_code= re.sub(r'^\n+','',clean_code)
      #lines of code
      matches=re.findall('\n',clean_code)

      self.clean_code=clean_code
      self.loc=len(matches)

    def analyze_code(self):
        # Analyze the code and populate the variables accordingly
        self.analyze_method()
        self.number_of_methods = len(self.method_name)

        self.analyze_loops()
        self.no_of_loops = len(self.loops)

        self.analyze_control_structures()
        self.no_of_controlStruct = len(self.controlStruct)

        self.analyze_variables()
        self.no_of_variables=len(self.variable_name)

        self.analyze_nesting_depth()
        self.find_loc()





In [ ]:
def buildingDF(value):

  
  classification.append(value)


In [ ]:
import glob
def file_access():
  files = glob.glob('/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/*', recursive = True)
  for file in files:

    original_path=glob.glob(file+'/original/*', recursive = True)
    og_file=open(original_path,'r')
    original_code = CodeAnalyzer(og_file.read())
    original_code.analyze_code()

    non_plag=glob.glob(file+'/non-plagiarized/*', recursive = True)
    for non_plag_folders in non_plag:
      non_plag_path=glob.glob(non_plag_folders+'/*',recursive=True)
      #non_plag_code stores required file location

      nplag_file=open(non_plag_path,'r')
      non_plagiarized_code = CodeAnalyzer(nplag_file.read())
      non_plagiarized_code.analyze_code()

      buildingDF(0) #0 implies non-plagiarized

    plagiarized=glob.glob(file+'/plagiarized/*', recursive = True)
    for plagiarized_folders in plagiarized:
      plagiarized_code_parent_folder=glob.glob(plagiarized_folders+'/*', recursive = True)
      for plagiarized_code_folder in plagiarized_code_parent_folder:
        plagiarized_path=glob.glob(plagiarized_code_folder+'/*', recursive = True)
        #plagiarized_code stores required file location

        plag_file=open(plagiarized_path,'r')
        plagiarized_code = CodeAnalyzer(plag_file.read())
        plagiarized_code.analyze_code()

        buildingDF(1) #1 implies plagiarized


In [ ]:
import pandas as pd

df=pd.DataFrame(columns=['Diff in LOC','Diff in No. of Loops','Cosine_Similarity','Class'])

dloc=[]
dnol=[]
classification=[]

file_access()


df['Diff in LOC']=dloc
df['Diff in No. of Loops']=dnol
df['Class']=classification

In [104]:
#testing class

original_path="/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-03/original/T3.java"

og_file=open(original_path,'r')
original_code = CodeAnalyzer(og_file.read())
original_code.analyze_code()

In [116]:
plagiarized_path="/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-03/plagiarized/L2/02/Main.java"

plag_file=open(plagiarized_path,'r')
plagiarized_code = CodeAnalyzer(plag_file.read())
plagiarized_code.analyze_code()

In [145]:
print(original_code.variable_name)
print(plagiarized_code.variable_name)

['input', 'weight', 'feet', 'inches', 'height', 'bmi']
['sc', 'weightUser', 'feetUser', 'inchesUser', 'heightUser', 'bmi']


In [147]:
print(original_code.loc)
print(plagiarized_code.loc)

27
30


In [148]:
print(original_code.method_return_type)
print(plagiarized_code.method_return_type)

[None]
[None]


In [143]:
original_code.method_name

['main']